## get user credentials

In [1]:
import os
import re
import json
import getpass
from pathlib import Path

STORAGE_DIR = "cv_storage"
EMAIL_REGEX = r"^[\w\.-]+@[a-zA-Z\d\.-]+\.[a-zA-Z]{2,}$"


def sanitize_email(email: str) -> str:
    return re.sub(r"[^\w\.-]", "_", email.lower())


def setup_environment():
    """
    Prompts for:
      - USER_EMAIL      (your email for CV storage & sending)
      - GMAIL_APP_PASSWORD
    Saves them in a user-specific credentials.json under cv_storage/{user_dir}
    """
    while True:
        user_email = input("📧 Enter your email address (for CV storage & sending): ").strip().lower()
        if re.match(EMAIL_REGEX, user_email):
            break
        print("❌ Invalid email format. Use name@domain.com")

    safe_email = sanitize_email(user_email)
    user_dir = Path(STORAGE_DIR) / safe_email
    user_dir.mkdir(parents=True, exist_ok=True)
    credentials_path = user_dir / "credentials.json"

    if credentials_path.exists():
        update = input("⚠️ Credentials already exist. Do you want to update them? (y/N): ").strip().lower()
        if update != "y":
            print("✅ Keeping existing credentials.")
            return

    while True:
        gmail_app_pwd = getpass.getpass("🔑 Enter your Gmail App Password (for SMTP): ").strip()
        if gmail_app_pwd:
            break
        print("❌ Password cannot be empty.")

    credentials = {
        "USER_EMAIL": user_email,
        "GMAIL_APP_PASSWORD": gmail_app_pwd
    }

    with open(credentials_path, "w") as f:
        json.dump(credentials, f, indent=2)

    print(f"✅ Credentials saved to {credentials_path}")


def get_user_email() -> tuple[str, str]:
    """
    Loads email from stored credentials.json
    Returns (email, user_dir)
    """
    for user_folder in Path(STORAGE_DIR).iterdir():
        credentials_path = user_folder / "credentials.json"
        if credentials_path.exists():
            with open(credentials_path) as f:
                credentials = json.load(f)
                user_email = credentials.get("USER_EMAIL")
                if user_email:
                    return user_email, str(user_folder)

    raise RuntimeError("❌ No valid credentials found. Run setup_environment() first.")


## llm config

In [2]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")


from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # grok_api_key='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'
    # other params...
)



# job extractor

In [3]:
from langchain_core.prompts import PromptTemplate

def extract_job_and_contact_info(job_paragraph: str) -> str:
    prompt_extract = PromptTemplate.from_template("""
### JOB POSTING ANALYSIS TASK
Analyze this job posting and extract structured information:

{job_paragraph}

### OUTPUT REQUIREMENTS:
- Return JSON with two root keys: "employer_info" and "position_details"
- Output ONLY valid JSON - no commentary
- Include fields ONLY when explicitly mentioned
- Maintain original wording for values
- Handle all job types (medical, tech, education, etc.)

1. EMPLOYER_INFO (hiring organization/individual):
{{
    "full_name": "(if individual)",
    "organization": "(company/institution name)",
    "department": "(specific division/team)",
    "industry": "(medical, tech, education, etc.)",
    "contact": {{
        "email": "(preferred)",
        "phone": "(if provided)",
        "website": "(career portal/LinkedIn)"
    }},
    "location": {{
        "city": "(primary workplace)",
        "country": "(if mentioned)",
        "remote_options": "(hybrid/remote flags)"
    }},
    "organization_type": "(hospital, startup, university, etc.)",
    "key_attributes": ["list", "of", "notable", "features"]
}}

2. POSITION_DETAILS (job characteristics):
{{
    "title": "(official job name)",
    "type": "(full-time, contract, internship)",
    "category": "(clinical, engineering, research, etc.)",
    "level": "(junior, senior, principal)",
    "salary": {{
        "range": "(numbers or description)",
        "currency": "(if specified)",
        "bonuses": "(signing/performance bonuses)"
    }},
    "requirements": {{
        "education": "(degrees/certifications)",
        "experience": "(years/type)",
        "skills": ["technical", "and", "soft", "skills"],
        "licenses": "(industry-specific certifications)"
    }},
    "responsibilities": ["list", "of", "core", "duties"],
    "benefits": ["healthcare", "retirement", "perks"],
    "deadlines": {{
        "application": "(DD/MM/YYYY or relative)",
        "start_date": "(if specified)"
    }},
    "travel_requirements": "(percentage or description)",
    "reporting_structure": "(supervisory relationships)",
    "performance_metrics": "(KPIs/success measures)"
}}

### OUTPUT ONLY VALID JSON WITH NO MARKDOWN FORMATTING
""")
    chain_extract = prompt_extract | llm
    # prompt = prompt_extract.format(job_paragraph=job_paragraph)
    response = chain_extract.invoke({'job_paragraph': job_paragraph})
    return response.content





import re

def validate_or_ask_email(extracted_json: dict) -> str:
    # Check nested contact info first, then fallback to manual input
    email = extracted_json.get("employer_info", {}).get("contact", {}).get("email")
    
    if email and re.match(r"^[\w\.-]+@[a-zA-Z\d\.-]+\.[a-zA-Z]{2,}$", email):
        return email
    
    # If no valid email found, prompt user
    print("\n✉️ No valid email found in job posting. Please provide one:")
    while True:
        manual_email = input("Contact email: ").strip()
        if re.match(r"^[\w\.-]+@[a-zA-Z\d\.-]+\.[a-zA-Z]{2,}$", manual_email):
            return manual_email
        print("Invalid format. Please enter a valid email address (e.g. name@company.com)")
        




from langchain_core.output_parsers import JsonOutputParser



def run_outreach_intake():
    global cv_data
    print("📋 Paste the job posting below:")
    job_text = input("Job Posting: ")

    print("\n🔍 Extracting details using LLM...")
    ext = extract_job_and_contact_info(job_text)
    parser = JsonOutputParser()
    extracted = parser.parse(ext)
    print("\n✅ Extracted JSON:\n", extracted)

    extracted_json = extracted
    cv_data = extracted
    
    # Get nested contact information
    contact_info = extracted_json.get("employer_info", {}).get("contact", {})
    contact_email = validate_or_ask_email({
        "contact_email": contact_info.get("email"),
        **extracted_json
    })

    print("\n📌 Final Info:")
    print(f"Industry: {extracted_json.get('employer_info', {}).get('industry', 'N/A')}")
    print(f"Organization: {extracted_json.get('employer_info', {}).get('organization', 'N/A')}")
    print("Contact Email:", contact_email)
    print("\nPosition Details:")
    for key, value in extracted_json.get("position_details", {}).items():
        print(f"- {key.replace('_', ' ').title()}: {value or 'N/A'}")

    return {
        "employer_info": extracted_json.get("employer_info", {}),
        "position_details": extracted_json.get("position_details", {}),
        "contact_email": contact_email,
        
    }
    







## cv details extractor

In [4]:

import datetime
import re
import os
import json
from pathlib import Path
from typing import List, Dict, Optional
from langchain_community.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
import getpass

# Enhanced configuration
STORAGE_DIR = "cv_storage"
EMAIL_REGEX = r"^[\w\.-]+@[a-zA-Z\d\.-]+\.[a-zA-Z]{2,}$"
# os.makedirs(STORAGE_DIR, exist_ok=True)








def upload_cv(user_dir: str) -> str:
    """Handle CV upload with comprehensive validation"""
    while True:
        file_path = input("📄 Enter CV path (.pdf/.docx): ").strip()
        
        if not os.path.isfile(file_path):
            print("❌ File not found")
            continue
            
        try:
            if file_path.lower().endswith(".pdf"):
                loader = PyPDFLoader(file_path)
            elif file_path.lower().endswith(".docx"):
                loader = UnstructuredWordDocumentLoader(file_path)
            else:
                print("❌ Unsupported format. Use PDF/DOCX")
                continue

            documents = loader.load()
            text = "\n".join([doc.page_content for doc in documents])
            
            # Save original file
            safe_filename = re.sub(r"[^\w\.-]", "_", os.path.basename(file_path))
            save_path = os.path.join(user_dir, safe_filename)
            Path(save_path).write_bytes(Path(file_path).read_bytes())
            
            print(f"✅ CV saved: {save_path}")
            return text
            
        except Exception as e:
            print(f"🚨 Error processing file: {str(e)}")



class ProfessionalProfile(BaseModel):
    """Universal professional profile model"""
    full_name: str = Field(..., description="Full legal name")
    contact_email: str = Field(..., description="Primary contact email")
    phone: Optional[str] = Field(None, description="Contact phone number")
    summary: Optional[str] = Field(None, description="Professional summary")
    
    education: List[Dict] = Field(
        default_factory=list,
        description="List of educational achievements with degrees, institutions, and dates if provided"
    )
    
    experience: List[Dict] = Field(
        default_factory=list,
        description="Work history with job titles, companies, dates  if provided, and key achievements"
    )
    
    technical_skills: List[str] = Field(
        default_factory=list,
        description="Technical skills relevant to the industry"
    )
    
    certifications: List[str] = Field(
        default_factory=list,
        description="Professional certifications and licenses"
    )
    
    projects: List[Dict] = Field(
        default_factory=list,
        description="Notable projects with descriptions and outcomes"
    )
    
    industry_preferences: List[str] = Field(
        default_factory=list,
        description="Preferred industries or sectors"
    )



parser = JsonOutputParser(pydantic_object=ProfessionalProfile)

prompt = PromptTemplate(
    template="""**Professional Profile Analysis Task**
Act as an expert career analyst with deep knowledge across industries (tech, healthcare, finance, engineering). 
Extract structured information while identifying transferable skills and cross-domain competencies.
"Please extract the following fields from the CV and return them in JSON format without any preamble: ..."

**Fields to Extract:**
- full_name
- contact_email
- phone
- summary
- linkedin
- github
- education (list)
- experience (list)
- technical_skills (list)
- soft_skills (list, optional)
- certifications (list)
- projects (list)
- languages (optional)
YOU CAN ADD AND SUBTRACT FIELDS ACCORDING TO PROVIDED CV AND INDUSTRY

**Analysis Guidelines:**
1. Core Identification:
- Extract full legal name from header/contact section
- Verify email format (name@domain.tld)
- Identify phone numbers in international format (+XXX...)
- Extract summary
- linked in link(if provided)
- github link(if provided)

2. Education Analysis:
- Parse degrees with majors/specializations
- Flag accreditation status for institutions
- Convert dates to MM/YYYY format  if provided
- Highlight research projects/theses


3. Experience Processing: 
- Separate employment history from internships
- Identify technical/soft skill development
- Quantify achievements ("Increased X by Y%")
- Map technologies to industry standards

4. Skill Extraction:
- Categorize skills:
  • Technical (tools/platforms)
  • Methodologies (Agile, Six Sigma)
  • Domain Knowledge (HIPAA, GAAP)
- Identify skill maturity levels:
  (Beginner < 1yr, Intermediate 1-3yr, Expert 3+yr)

5. Cross-Industry Transfer Analysis:
- Identify portable competencies between industries
- Highlight leadership/management patterns
- Extract crisis management evidence
- Flag multilingual capabilities

**Structured Output Requirements:**
{format_instructions}

**Content Processing Rules:**
- Preserve original wording unless ambiguous
- only if starting date is provided Convert relative dates ("current" → {today})
- Expand acronyms first occurrence (WHO → World Health Organization)
- Handle conflicting info (prioritize most recent)


**Input Profile:**
{text}""",
    input_variables=["text"],
    partial_variables={
        "format_instructions": parser.get_format_instructions(),
        "today": datetime.date.today().strftime("%m/%Y")
    },
)


def parse_cv(text: str) -> dict:
    """Process CV text through LLM parsing chain"""
    try:
        chain = prompt | llm | parser
        result = chain.invoke({"text": text})
        return dict(result)
    except Exception as e:
        print(f"⚠️ Error parsing CV: {str(e)}")
        return {}

def save_parsed_data(data: dict, user_dir: str) -> None:
    """Save structured profile data"""
    save_path = Path(user_dir) / "profile_data.json"
    with open(save_path, "w") as f:
        json.dump(data, f, indent=2)
    print(f"📄 Profile data saved to {save_path}")

def display_profile_summary(user_dir: str) -> None:
    """Display formatted profile summary"""
    data_file = Path(user_dir) / "profile_data.json"
    if not data_file.exists():
        print("ℹ️ No profile data available")
        return
        
    with open(data_file) as f:
        data = json.load(f)
    
    print("\n🌟 Professional Summary:")
    print(f"Name: {data.get('full_name', 'N/A')}")
    print(f"Contact: {data.get('contact_email', 'N/A')} | {data.get('phone', 'N/A')}")
    print(f"\n🏫 Education ({len(data['education'])} entries)")
    print(f"\n💼 Experience ({len(data['experience'])} positions)")
    print(f"\n🛠️ Technical Skills ({len(data['technical_skills'])} listed)")

def merge_with_llm(existing: dict, new: dict) -> dict:
    """Use LLM to intelligently merge two structured profile dicts."""
    if not existing:
        return new
    if not new:
        return existing

    try:
        prompt_text = f"""
You are a helpful assistant tasked with merging two structured professional profiles extracted from CVs. 
Your goal is to intelligently combine the data from both profiles, avoiding redundancy, preserving the most complete and informative entries, and resolving conflicts sensibly.

Act as an expert career analyst with deep cross-industry knowledge (tech, healthcare, finance, engineering). 
You must identify transferable skills, merge overlapping entries, and preserve all unique information, especially for certifications.

Please extract and return the following fields in raw JSON format **only**, without preamble or commentary.

---
**Fields to Extract:**
- full_name
- contact_email
- phone
- summary
- linkedin
- github
- education (list)
- experience (list)
- technical_skills (list)
- soft_skills (list, optional)
- certifications (list)
- projects (list)
- languages (optional)
YOU CAN ADD AND SUBTRACT FIELDS ACCORDING TO PROVIDED CV AND INDUSTRY
---

**Guidelines for Merging and Extraction:**

1. **Core Info:**
   - Extract full legal name from the header or contact block.
   - Emails must be valid (e.g., name@domain.com).
   - Phone numbers must be in international format (+XXX...).
   - Include LinkedIn and GitHub links if found.

2. **Education:**
   - List all degrees and specializations.
   - Include institution name, degree, field, start and end date (MM/YYYY).
   - Highlight research projects or thesis titles if available.
   - Avoid duplication; if same degree exists with more details, keep the more complete version.

3. **Experience:**
   - Distinguish jobs, internships, freelance, and volunteering.
   - Include job title, company, duration, technologies used, and quantifiable outcomes.
   - Keep the most recent or complete version of similar roles.
   - Use consistent date format (MM/YYYY).

4. **Skills:**
   - Group skills into:
     • Technical Skills (tools, platforms, libraries)
     • Methodologies (Agile, Scrum, Six Sigma)
     • Domain Knowledge (e.g., GDPR, HIPAA)
   - Include experience levels if stated (e.g., Expert, Intermediate).

5. **Certifications (✅ Important):**
   - Extract each certification with full name, issuing organization, and date (if available).
   - Only merge certifications if the **exact full name and issuer match**.
   - If titles are slightly different or have extra info (e.g., "AWS Certified Developer – Associate" vs "AWS Developer Cert"), treat them as separate and preserve both.
    
6. **Projects:**
   - Include title, description, technologies used, role (if specified), and duration.
   - Projects may come from personal work, hackathons, university, or freelance.
   - Merge only if titles and descriptions are identical or nearly identical.
   - Preserve all distinct projects — no limit.
7. **Languages (if any):**
   - Include spoken languages and proficiency if listed.

8. **General Rules:**
   - Avoid redundancy and merge smartly.
   - Prioritize clarity, structure, and richness of information.
   - Do not add placeholder or fabricated data.
   - Output should be a valid JSON object.
   

---

**Input Profiles:**

Profile A (existing):
{json.dumps(existing, indent=2)}

Profile B (newly parsed):
{json.dumps(new, indent=2)}

Return ONLY the merged profile in raw JSON format.
Do NOT include explanations or commentary. Just output the final merged JSON.
"""
    #     response = llm.invoke(prompt_text)
    #     return json.loads(response.content.strip())
    # except Exception as e:
    #     print(f"⚠️ LLM merge failed: {e}")
    #     return {**existing, **new}  # fallback: simple merge
        response = llm.invoke(prompt_text)
        raw_output = response.content.strip()

        # 📌 Safely extract the JSON part from the output
        json_start = raw_output.find('{')
        json_end = raw_output.rfind('}')
        if json_start == -1 or json_end == -1:
            raise ValueError("No JSON object found in LLM output")

        clean_json = raw_output[json_start:json_end+1]
        return json.loads(clean_json)

    except Exception as e:
        print(f"⚠️ LLM merge failed: {e}")
        print(f"⚠️ Raw LLM output:\n{response.content if 'response' in locals() else 'No response'}")
        return {**existing, **new}  # fallback

def run_cv_pipeline(user_dir: str) -> None:
    """Main execution flow"""
    if not any(Path(user_dir).iterdir()):
        print("📥 Initial profile setup")
        cv_text = upload_cv(user_dir)
        parsed_data = parse_cv(cv_text)
        save_parsed_data(parsed_data, user_dir)
    else:
        manage_cv(user_dir)
        

def manage_cv(user_dir: str) -> None:
    """Enhanced CV management system"""
    while True:
        action = input("\nChoose: [U]pload new, [D]elete, [V]iew, [E]xit: ").strip().lower()
        
        # if action == "u":
        #     text = upload_cv(user_dir)
        #     parsed = parse_cv(text)
        #     save_parsed_data(parsed, user_dir)
        if action == "u":
            text = upload_cv(user_dir)
            new_parsed = parse_cv(text)

            # Load existing data if it exists
            existing_file = Path(user_dir) / "profile_data.json"
            if existing_file.exists():
                with open(existing_file) as f:
                    existing_parsed = json.load(f)
                merged_data = merge_with_llm(existing_parsed, new_parsed)
                print("🔄 Merged new CV with existing profile using LLM.")
            else:
                merged_data = new_parsed
                print("🆕 No existing profile found. Saving new data.")

            save_parsed_data(merged_data, user_dir)

        elif action == "d":
            confirm = input("⚠️ Delete ALL profile data? (y/n): ").lower()
            if confirm == "y":
                for item in Path(user_dir).glob("*"):
                    item.unlink()
                print("🗑️ Profile data deleted")
        elif action == "v":
            display_profile_summary(user_dir)
        elif action == "e":
            break
        else:
            print("❌ Invalid option")




## custom cv maker


In [5]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import Runnable
from typing import Dict, Literal
from pathlib import Path
import json
from fpdf import FPDF
import re

def clean_unicode(text: str) -> str:
    replacements = {
        '\u2013': '-',  # en dash
        '\u2014': '-',  # em dash
        '\u2018': "'",  # left single quote
        '\u2019': "'",  # right single quote
        '\u201c': '"',  # left double quote
        '\u201d': '"',  # right double quote
        '\u2026': '...',  # ellipsis
    }
    for bad, good in replacements.items():
        text = text.replace(bad, good)
    return text

def generate_custom_cv(
    user_dir: str,
    job_posting_info: Dict,
    llm,
    action: Literal["preview", "save", "edit"] = "preview",
    edit_instructions: str = ""
) -> str:
    # Load user profile
    existing_file = Path(user_dir) / "profile_data.json"
    if not existing_file.exists():
        raise FileNotFoundError("User profile_data.json not found.")

    with open(existing_file, "r") as f:
        user_data = json.load(f)

    generated_cv_path = Path(user_dir) / "latest_cv.txt"
    print(user_data)
    print(cv_data)
    # Main CV generation
    if action == "preview":
        # Initial prompt
        prompt = PromptTemplate(
        input_variables=["user_data", "job_posting_info"],
        template="""
You are an expert resume writer specializing in ATS-optimized, role-targeted CVs.

Generate a clean, plain text CV using this structure:

[Full Name]
[Email] | [Phone] | [LinkedIn/GitHub if available]

SUMMARY:
- 1-2 sentence professional summary with 3 keywords from job description

EXPERIENCE:
[Job Title] - [Company], [Location]
[MM/YYYY - MM/YYYY]
- CAR-formatted bullet points (Challenge-Action-Result)
- Include relevant technical keywords
- Quantify achievements where possible

EDUCATION:
[Degree] - [Institution]
[MM/YYYY - MM/YYYY]

SKILLS:
- Technical skills matching job requirements
- Tools/platforms from job description

PROJECTS (if relevant to position):
[Project Name]
- Brief description showing job-relevant skills

---

Candidate Info:
{user_data}

Job Posting Details:
{job_posting_info}

---
"""
)
        chain: Runnable = prompt | llm
        try:
            result = chain.invoke({
                "user_data": json.dumps(user_data, indent=2),
                "job_posting_info": json.dumps(job_posting_info, indent=2),
            })
        except Exception as e:
            print("❌ Error during LLM invocation:", e)
            return "Failed to generate CV. Check the logs."

        if isinstance(result, str):
            cv_text = clean_unicode(result.strip())
        elif isinstance(result, dict) and 'content' in result:
            cv_text = clean_unicode(result['content'].strip())
        elif hasattr(result, 'content'):
            cv_text = clean_unicode(result.content.strip())
        else:
            raise ValueError("Unexpected LLM result format.")

        generated_cv_path.write_text(cv_text, encoding="utf-8")
        return cv_text

    elif action == "edit":
        if not generated_cv_path.exists():
            raise FileNotFoundError("Preview the CV first before editing.")
        
        previous_cv = generated_cv_path.read_text(encoding="utf-8")

        edit_prompt = PromptTemplate(
            input_variables=['previous_cv','edit_instructions',   'user_data', 'job_posting_info'],
            template="""
You are a professional resume editor.

Here is the current CV:
---
{previous_cv}
---

Here are the instructions from the user:
---
{edit_instructions}
---


📄 Candidate Info:
---
{user_data}
---
🧾 Job Posting:
---
{job_posting_info}
---

Revise the CV accordingly while preserving the ATS-optimized formatting.
"""
        )
        chain: Runnable = edit_prompt | llm
        result = chain.invoke({
            "previous_cv": previous_cv,
            "edit_instructions": edit_instructions,
            "user_data": json.dumps(user_data, indent=2),
            "job_posting_info": json.dumps(job_posting_info, indent=2),
            
        })
        cv_text = clean_unicode(result.content.strip())
        generated_cv_path.write_text(cv_text, encoding="utf-8")
        return cv_text

    elif action == "save":
        if not generated_cv_path.exists():
            raise FileNotFoundError("Preview the CV first before saving.")
        cv_text = generated_cv_path.read_text(encoding="utf-8")

        output_path = Path(user_dir) / "generated_cv.pdf"
        pdf = FPDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()
        pdf.set_font("Arial", size=11)
        for line in cv_text.split("\n"):
            pdf.multi_cell(0, 10, txt=line.strip())

        pdf.output(str(output_path))
        return f"PDF saved at: {output_path.resolve()}"

    else:
        raise ValueError("Invalid action. Use 'preview', 'edit', or 'save'.")


def run_cv_interactive_flow(user_dir: str, job_posting_info: Dict, llm):
    print("Generating preview...")
    preview = generate_custom_cv(user_dir, job_posting_info, llm, action="preview")
    print("\n📄 Preview of Generated CV:\n")
    print(preview)

    while True:
        user_action = input("\nWhat would you like to do next? [save/edit/exit]: ").strip().lower()

        if user_action == "save":
            result = generate_custom_cv(user_dir, job_posting_info, llm, action="save")
            print(result)
            break

        elif user_action == "edit":
            instructions = input("Enter your editing instructions for the CV:\n")
            edited = generate_custom_cv(user_dir, job_posting_info, llm, action="edit", edit_instructions=instructions)
            print("\n📝 Updated CV:\n")
            print(edited)

        elif user_action == "exit":
            print("Exiting without saving.")
            break

        else:
            print("❌ Invalid input. Please choose: save / edit / exit")



## cover letter


In [6]:
def generate_cover_letter(
    user_dir: str,
    job_posting_info: Dict,
    llm,
    action: Literal["preview", "edit"] = "preview",
    edit_instructions: str = ""
) -> str:
    profile_path = Path(user_dir) / "profile_data.json"
    if not profile_path.exists():
        raise FileNotFoundError("User profile_data.json not found.")

    with open(profile_path, "r") as f:
        user_data = json.load(f)

    generated_letter_path = Path(user_dir) / "latest_cover_letter.txt"

    if action == "preview":
        prompt = PromptTemplate(
            input_variables=["user_data", "job_posting_info"],
            template="""
You are an expert in writing tailored, concise, and professional cover letters for job applications.

Use the following data to generate a compelling, personalized cover letter.

---

👤 **Candidate Profile**
{user_data}

🧾 **Job Posting**
{job_posting_info}

---

✍️ **Instructions**:
- Use standard business letter format: Address, Greeting, 3–4 short paragraphs, and Signature.
- Keep it under 350 words.
- Address the hiring manager if possible ("Dear Hiring Manager" if not known).
- Mention 2–3 core skills or experiences that match the job.
- End with a polite call to action.
- Use clear, professional language.

Now generate the cover letter.
"""
        )

        chain: Runnable = prompt | llm
        result = chain.invoke({
            "user_data": json.dumps(user_data, indent=2),
            "job_posting_info": json.dumps(job_posting_info, indent=2),
        })

        letter_text = clean_unicode(result.content.strip())
        generated_letter_path.write_text(letter_text, encoding="utf-8")
        return letter_text

    elif action == "edit":
        if not generated_letter_path.exists():
            raise FileNotFoundError("Preview the letter first before editing.")

        previous_letter = generated_letter_path.read_text(encoding="utf-8")

        edit_prompt = PromptTemplate(
            input_variables=["previous_letter", "edit_instructions"],
            template="""
You are a professional writing assistant.

Here's the current cover letter:
---
{previous_letter}
---

Apply the following editing instructions:
---
{edit_instructions}
---

Return the updated cover letter.
"""
        )
        chain: Runnable = edit_prompt | llm
        result = chain.invoke({
            "previous_letter": previous_letter,
            "edit_instructions": edit_instructions,
        })

        letter_text = clean_unicode(result.content.strip())
        generated_letter_path.write_text(letter_text, encoding="utf-8")
        return letter_text

    else:
        raise ValueError("Invalid action. Use 'preview' or 'edit'.")


def run_cover_letter_interactive_flow(user_dir: str, job_posting_info: Dict, llm):
    print("\n✉️ Generating Cover Letter Preview...\n")
    preview = generate_cover_letter(user_dir, job_posting_info, llm, action="preview")
    print(preview)

    while True:
        user_action = input("\nWhat would you like to do with the cover letter? [edit/exit]: ").strip().lower()

        if user_action == "edit":
            instructions = input("Enter your editing instructions for the cover letter:\n")
            updated = generate_cover_letter(user_dir, job_posting_info, llm, action="edit", edit_instructions=instructions)
            print("\n📝 Updated Cover Letter:\n")
            print(updated)

        elif user_action == "exit":
            print("Finished with cover letter.")
            break

        else:
            print("❌ Invalid input. Choose 'edit' or 'exit'")


## email sending

In [7]:
def send_email_with_cv(user_dir: str, recipient_email: str):
    from pathlib import Path
    import json
    import os
    from email.message import EmailMessage
    import smtplib, ssl

    user_dir = Path(user_dir)

    # Load credentials
    with open(user_dir / "credentials.json") as f:
        creds = json.load(f)

    sender_email = creds["USER_EMAIL"]
    app_password = creds["GMAIL_APP_PASSWORD"]

    # Get cover letter
    cover_letter_path = user_dir / "latest_cover_letter.txt"
    if not cover_letter_path.exists():
        raise FileNotFoundError(f"❌ Cover letter not found at {cover_letter_path}")
    cover_letter_text = cover_letter_path.read_text()

    # Get generated CV
    pdf_path = user_dir / "generated_cv.pdf"
    if not pdf_path.exists():
        raise FileNotFoundError(f"❌ Generated CV not found at {pdf_path}")

    # Compose email
    msg = EmailMessage()
    msg["Subject"] = f"Regarding Job Application – {sender_email.split('@')[0].title()}"
    msg["From"] = sender_email
    msg["To"] = recipient_email
    msg.set_content(cover_letter_text)

    # Attach CV
    pdf_data = pdf_path.read_bytes()
    msg.add_attachment(pdf_data, maintype='application', subtype='pdf', filename=pdf_path.name)

    # Send via Gmail SMTP
    context = ssl.create_default_context()
    print("📡 Connecting to Gmail SMTP...")
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as smtp:
        print("🔐 Logging in...")
        smtp.login(sender_email, app_password)
        print("📤 Sending email...")
        smtp.send_message(msg)
        print("✅ Email sent")

   

    print(f"✅ Email sent to {recipient_email} with CV: {pdf_path.name}")


## run the code

In [8]:

if __name__ == "__main__":
    # 1) Prompt once (or load existing) for all creds including GROQ_API_KEY
    setup_environment()

    # 2) Now you can just pull from env anywhere
    import os
    from dotenv import load_dotenv
    load_dotenv()
    # 3) Get your user email & dir
    email, user_dir = get_user_email()
    print(f"📂 Your directory: {user_dir}")
    print(f"📧 Your email: {email}")

    # 4) Continue with the rest of your pipeline...
    run_cv_pipeline(user_dir)
    run_outreach_intake()
    run_cv_interactive_flow(user_dir, cv_data, llm)
    run_cover_letter_interactive_flow(user_dir, cv_data, llm)
    send_email_with_cv(user_dir=user_dir, recipient_email="shahmeergull20@gmail.com")
    
    

✅ Keeping existing credentials.
📂 Your directory: cv_storage\shahmeergull20_gmail.com
📧 Your email: shahmeergull20@gmail.com
📋 Paste the job posting below:

🔍 Extracting details using LLM...

✅ Extracted JSON:
 {'employer_info': {'organization': 'Es Magico AI Studio', 'industry': 'tech', 'contact': {'email': 'karan.trehan@esmagico.in'}, 'location': {'city': 'Mumbai', 'remote_options': 'hybrid'}, 'organization_type': 'startup', 'key_attributes': []}, 'position_details': {'title': 'Intern', 'type': 'full-time & paid internship', 'category': 'AI', 'level': '', 'salary': {'range': '', 'currency': '', 'bonuses': ''}, 'requirements': {'education': 'BTech / BCA / MTech / MCA / BSc / etc.', 'experience': '', 'skills': ['Python scripting', 'AI (RAGs, Agents, Prompting)'], 'licenses': ''}, 'responsibilities': [], 'benefits': [], 'deadlines': {'application': '', 'start_date': ''}, 'travel_requirements': 'thrice a week', 'reporting_structure': '', 'performance_metrics': ''}}

📌 Final Info:
Industr